In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## NOT USED


In [ ]:
df = pd.read_csv("../bookrecommendor_data/Books_Preprocessed.csv")
df = df[["ISBN", "Title", "Author", "Year", "Publisher", "Processed_Title"]].dropna()
df.columns = ["ISBN", "Title", "Author", "Year", "Publisher", "Processed_Title"]
df = df.drop_duplicates(subset=["Title"])

In [ ]:
df["Title", "Author", "Proce"]

,ISBN,Title,Author,Year,Publisher,Processed_Title
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,classical mythology
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,clara callan
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,decision normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,flu story great influenza pandemic 1918 search...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,mummy urumchi
...,...,...,...,...,...,...
242145,014002803X,Anti Death League,Kingsley Amis,1975,Viking Press,anti death league
242146,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,flashpoint promise peril new world
242148,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,lily dale true story town talk dead
242149,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,republic world classic


In [2]:
from datasets import load_dataset

ds = load_dataset("booksouls/goodreads-book-descriptions")

c:\Users\silve\anaconda3\envs\nlp-project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pd.DataFrame(ds)

,train
0,"{'title': 'Good Harbor', 'description': 'Anita..."
1,{'title': 'The Unschooled Wizard (Sun Wolf and...
2,"{'title': 'Best Friends Forever', 'description..."
3,"{'title': 'The Aeneid for Boys and Girls', 'de..."
4,{'title': 'All's Fairy in Love and War (Avalon...
...,...
1021101,"{'title': 'The Spaniard's Blackmailed Bride', ..."
1021102,"{'title': 'Different Breeds', 'description': '..."
1021103,"{'title': 'This Sceptred Isle, Vol. 10: The Ag..."
1021104,{'title': 'Sherlock Holmes and the July Crisis...


## USED DS


In [13]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'bertopic'

In [11]:
# Wczytaj dane
df = pd.read_csv("book_genre_prediction.csv")
df = df.dropna(subset=["title", "summary", "genre"])
df = df.drop_duplicates(subset=["title"])
df.head()

,index,title,genre,summary
0,0,Drowned Wednesday,fantasy,Drowned Wednesday is the first Trustee among ...
1,1,The Lost Hero,fantasy,"As the book opens, Jason awakens on a school ..."
2,2,The Eyes of the Overworld,fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,Magic's Promise,fantasy,The book opens with Herald-Mage Vanyel return...
4,4,Taran Wanderer,fantasy,Taran and Gurgi have returned to Caer Dallben...


In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(df['description'].tolist(), show_progress_bar=True)

NameError: name 'SentenceTransformer' is not defined

In [ ]:
topic_model = BERTopic(embedding_model=embedding_model)
topics, _ = topic_model.fit_transform(df['description'].tolist(), embeddings)

In [ ]:
df["topic"] = topics
df["embedding"] = list(embeddings)

In [ ]:
def recommend_books(input_title, top_n=5):
    input_idx = df[df["title"].str.lower() == input_title.lower()]
    if input_idx.empty:
        print("Tytuł nie znaleziony.")
        return []

    idx = input_idx.index[0]
    input_topic = df.loc[idx, "topic"]
    input_embedding = df.loc[idx, "embedding"]
    input_genre = df.loc[idx, "genre"]

    similarities = cosine_similarity([input_embedding], list(df["embedding"]))[0]

    scores = []
    for i in range(len(df)):
        score = 0
        if df.loc[i, "title"].lower() == input_title.lower():
            continue
        if input_title.split()[0].lower() in df.loc[i, "title"].lower():
            score += 2  # sim title
        if df.loc[i, "topic"] == input_topic:
            score += 2  # same title
        if df.loc[i, "genre"].lower() == input_genre.lower():
            score += 1  # same genre
        score += similarities[i]  # sim topic
        scores.append((i, score))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]
    recommendations = df.iloc[[i for i, _ in sorted_scores]][['title', 'genre', 'description']]

    return recommendations

In [ ]:
result = recommend_books("Drowned Wednesday")
print(result.to_string(index=False))